In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

# Add /src modules to path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import modules
from src.utils import DbConnector, haversine_np
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime, timedelta
from collections import Counter


In [16]:
# Connect to MySQL database

connector = DbConnector()
db = connector.db

You are connected to the database: mongodb
-----------------------------------------------



----

### Task 1
How many users, activities and trackpoints are there in the dataset?

In [17]:
collection = db["ActivityCollection"]

users_count = db.UserCollection.count_documents({})
activities_count = db.ActivityCollection.count_documents({})
trackpoints_count = db.ActivityCollection.aggregate(
    [{"$unwind": "$track_points"}, {"$count": "trackpoint_count"}]
)
trackpoints_count = next(trackpoints_count, {}).get("trackpoint_count", 0)


print(f"Total Users: {users_count}")
print(f"Total Activities: {activities_count}")
print(f"Total Trackpoints: {trackpoints_count}")

Total Users: 182
Total Activities: 15641
Total Trackpoints: 8501885


### Task 2
Find the average number of activities per user

In [18]:
average_activities_per_user = activities_count / users_count
print(f"Average Activities/User: {average_activities_per_user:.2f}")

Average Activities/User: 85.94


### Task 3
Find the top 20 users with the highest number of activities. 

In [19]:
users_activities_count = list(
    db["ActivityCollection"].aggregate(
        [
            {"$group": {"_id": "$user", "activity_count": {"$sum": 1}}},
            {"$sort": {"activity_count": -1}},
            {"$limit": 20},
        ]
    )
)

print("Top 20 Users with Most Activities:")

for user in users_activities_count:
    print(f"User: {user['_id']}, Activities: {user['activity_count']}")

Top 20 Users with Most Activities:
User: 085, Activities: 1089
User: 153, Activities: 977
User: 068, Activities: 920
User: 128, Activities: 876
User: 167, Activities: 810
User: 025, Activities: 715
User: 062, Activities: 538
User: 126, Activities: 420
User: 084, Activities: 411
User: 010, Activities: 402
User: 041, Activities: 399
User: 163, Activities: 354
User: 004, Activities: 346
User: 140, Activities: 345
User: 179, Activities: 305
User: 052, Activities: 282
User: 017, Activities: 265
User: 003, Activities: 261
User: 014, Activities: 236
User: 030, Activities: 210


### Task 4
Find all users who have taken a taxi. 

In [26]:
users_taken_taxi = db["ActivityCollection"].distinct(
    "user", {"transportation_mode": "taxi"}
)

print("Users who have taken a taxi:")
pprint(users_taken_taxi)

Users who have taken a taxi:
['010',
 '021',
 '052',
 '056',
 '058',
 '062',
 '065',
 '068',
 '075',
 '078',
 '080',
 '082',
 '084',
 '085',
 '098',
 '102',
 '105',
 '111',
 '114',
 '126',
 '128',
 '139',
 '153',
 '154',
 '161',
 '163',
 '167',
 '175',
 '179']


### Task 5
Find all types of transportation modes and count how many activities that are
tagged with these transportation mode labels. Do not count the rows where
the mode is null.

In [31]:
transportation_modes = list(
    db["ActivityCollection"].aggregate(
        [
            {"$group": {"_id": "$transportation_mode", "count": {"$sum": 1}}},
            {"$match": {"_id": {"$ne": None}}},
        ]
    )
)

print("Transportation Modes and Their Counts:")

transportation_modes.sort(key=lambda x: x["count"], reverse=True)
for mode in transportation_modes:
    print(f"Mode: {mode['_id']}, Count: {mode['count']}")

Transportation Modes and Their Counts:
Mode: walk, Count: 3927
Mode: bus, Count: 1820
Mode: bike, Count: 1519
Mode: car, Count: 751
Mode: subway, Count: 613
Mode: taxi, Count: 512
Mode: train, Count: 134
Mode: airplane, Count: 13
Mode: boat, Count: 7
Mode: run, Count: 4
Mode: motorcycle, Count: 2


### Task 6

#### a) Find the year with the most activities. 

In [35]:
year_most_activities = list(
    db.ActivityCollection.aggregate(
        [
            {"$project": {"year": {"$year": "$start_date_time"}}},
            {"$group": {"_id": "$year", "activity_count": {"$sum": 1}}},
            {"$sort": {"activity_count": -1}},
            {"$limit": 1},
        ]
    )
)[0]

print(f"Year with most activities: {year_most_activities['_id']}")
print(f"This year had {year_most_activities['activity_count']} activities")

Year with most activities: 2008
This year had 7861 activities


#### b) Is this also the year with most recorded hours?

This aggregation pipeline first calculates the number of hours for each activity by subtracting the start_date_time from the end_date_time (resulting in a duration in milliseconds) and then dividing by the number of milliseconds in an hour (3,600,000). It then groups the data by year and sums the total hours, sorts by total hours in descending order, and limits the results to the year with the highest number of hours.

In [33]:
year_most_hours = list(
    db.ActivityCollection.aggregate(
        [
            {
                "$project": {
                    "year": {"$year": "$start_date_time"},
                    "hours": {
                        "$divide": [
                            {"$subtract": ["$end_date_time", "$start_date_time"]},
                            3600000,  # Number of milliseconds in an hour
                        ]
                    },
                }
            },
            {"$group": {"_id": "$year", "total_hours": {"$sum": "$hours"}}},
            {"$sort": {"total_hours": -1}},
            {"$limit": 1},
        ]
    )
)[0]

print(
    f"Year with most recorded hours: {year_most_hours['_id']}, Hours: {year_most_hours['total_hours']:.2f}"
)

Year with most recorded hours: 2009, Hours: 9765.33


### Task 7
Find the total distance (in km) walked in 2008, by user with id=112.

In [37]:
user_id = "112"
activities_for_user = db.ActivityCollection.find({
    "user": user_id,
    "transportation_mode": "walk",
    "start_date_time": {"$gte": datetime(2008, 1, 1), "$lt": datetime(2009, 1, 1)}
})

total_distance = 0.0

for activity in activities_for_user:
    track_points = activity["track_points"]
    
    # Split track points into latitudes, longitudes for easier computation
    lats = np.array([point["latitude"] for point in track_points])
    longs = np.array([point["longitude"] for point in track_points])
    
    # Calculate the haversine distance for each consecutive track point and sum them up
    total_distance += np.sum(haversine_np(longs[:-1], lats[:-1], longs[1:], lats[1:]))

print(f"Total distance walked by user {user_id} in 2008: {total_distance:.2f} km")


Total distance walked by user 112 in 2008: 223.15 km


### Task 8
Find the top 20 users who have gained the most altitude meters.
- Output should be a field with (id, total meters gained per user).
- Remember that some altitude-values are invalid

In [44]:
def calculate_altitude_gain(track_points):
    """
    Calculate the altitude gain from a list of track points.
    """
    total_gain = 0
    for i in range(1, len(track_points)):
        altitude_difference = track_points[i]['altitude'] - track_points[i-1]['altitude']
        if altitude_difference > 0:
            total_gain += altitude_difference
    return total_gain

def top_users_by_altitude_gain():
    # Connect to the MongoDB instance
    activity_collection = db['ActivityCollection']

    user_altitude_gains = {}

    # Iterate over all activities in the ActivityCollection
    for activity in activity_collection.find():
        user = activity['user']
        altitude_gain = calculate_altitude_gain(activity['track_points'])

        # Convert altitude_gain from feet to meters
        altitude_gain *= 0.3048
        
        if user in user_altitude_gains:
            user_altitude_gains[user] += altitude_gain
        else:
            user_altitude_gains[user] = altitude_gain

    # Sort users by their total altitude gain
    sorted_users = sorted(user_altitude_gains.items(), key=lambda x: x[1], reverse=True)

    # Get top 20 users
    top_20_users = sorted_users[:20]

    for user, gain in top_20_users:
        print(f"User ID: {user}, Altitude Gain: {gain} meters")

top_users_by_altitude_gain()


User ID: 004, Altitude Gain: 332036.31839999993 meters
User ID: 128, Altitude Gain: 266359.42647999997 meters
User ID: 085, Altitude Gain: 259790.1839999999 meters
User ID: 041, Altitude Gain: 240768.86568000002 meters
User ID: 003, Altitude Gain: 233663.64240000013 meters
User ID: 030, Altitude Gain: 175679.70959999994 meters
User ID: 062, Altitude Gain: 152512.28832000014 meters
User ID: 039, Altitude Gain: 146703.59280000004 meters
User ID: 084, Altitude Gain: 132726.3792 meters
User ID: 167, Altitude Gain: 129899.2352000001 meters
User ID: 000, Altitude Gain: 121504.86239999998 meters
User ID: 002, Altitude Gain: 115198.24559999998 meters
User ID: 153, Altitude Gain: 112978.33135999988 meters
User ID: 025, Altitude Gain: 109158.57263999997 meters
User ID: 037, Altitude Gain: 99234.5894399998 meters
User ID: 140, Altitude Gain: 94846.29935999996 meters
User ID: 052, Altitude Gain: 81748.27440000004 meters
User ID: 017, Altitude Gain: 62581.35311999996 meters
User ID: 034, Altitude G

### Task 9
Find all users who have invalid activities, and the number of invalid activities
per user.
- An invalid activity is defined as an activity with consecutive trackpoints
where the timestamps deviate with at least 5 minutes. 

In [42]:
def has_invalid_activity(track_points):
    """
    Check if the activity has any consecutive trackpoints with a timestamp deviation of at least 5 minutes.
    """
    for i in range(1, len(track_points)):
        time_difference = track_points[i]['date_time'] - track_points[i-1]['date_time']
        if time_difference >= timedelta(minutes=5):
            return True
    return False

def users_with_invalid_activities():
    # Connect to the MongoDB instance
    activity_collection = db['ActivityCollection']

    user_invalid_activity_counts = {}

    # Iterate over all activities in the ActivityCollection
    for activity in activity_collection.find():
        user = activity['user']
        if has_invalid_activity(activity['track_points']):
            if user in user_invalid_activity_counts:
                user_invalid_activity_counts[user] += 1
            else:
                user_invalid_activity_counts[user] = 1

    # Filter for users with at least one invalid activity
    users_with_invalids = {user: count for user, count in user_invalid_activity_counts.items() if count > 0}

    for user, invalid_count in users_with_invalids.items():
        print(f"User ID: {user}, Invalid Activities: {invalid_count}")

users_with_invalid_activities()

User ID: 135, Invalid Activities: 5
User ID: 132, Invalid Activities: 3
User ID: 103, Invalid Activities: 24
User ID: 168, Invalid Activities: 19
User ID: 157, Invalid Activities: 9
User ID: 150, Invalid Activities: 16
User ID: 159, Invalid Activities: 5
User ID: 166, Invalid Activities: 2
User ID: 161, Invalid Activities: 2
User ID: 102, Invalid Activities: 9
User ID: 105, Invalid Activities: 14
User ID: 133, Invalid Activities: 4
User ID: 134, Invalid Activities: 31
User ID: 158, Invalid Activities: 9
User ID: 167, Invalid Activities: 135
User ID: 151, Invalid Activities: 1
User ID: 169, Invalid Activities: 9
User ID: 024, Invalid Activities: 27
User ID: 023, Invalid Activities: 11
User ID: 015, Invalid Activities: 46
User ID: 012, Invalid Activities: 43
User ID: 079, Invalid Activities: 2
User ID: 046, Invalid Activities: 13
User ID: 041, Invalid Activities: 201
User ID: 048, Invalid Activities: 1
User ID: 077, Invalid Activities: 3
User ID: 083, Invalid Activities: 15
User ID: 084,

### Task 10
Find the users who have tracked an activity in the Forbidden City of Beijing.
- In this question you can consider the Forbidden City to have
coordinates that correspond to: lat 39.916, lon 116.397.

In [45]:
coordinates = {"latitude": 39.916, "longitude": 116.397}

pipeline = [
    {"$unwind": "$track_points"},
    {
        "$match": {
            "track_points.latitude": {
                "$gte": coordinates["latitude"] - 0.001,
                "$lte": coordinates["latitude"] + 0.001,
            },
            "track_points.longitude": {
                "$gte": coordinates["longitude"] - 0.001,
                "$lte": coordinates["longitude"] + 0.001,
            },
        }
    },
    {"$group": {"_id": "$user"}},
]

output = db.ActivityCollection.aggregate(pipeline)
output = [user["_id"] for user in output]
output

['153', '131', '004', '018', '019']

### Task 11
Find all users who have registered transportation_mode and their most used
transportation_mode. 
- The answer should be on format (user_id,
most_used_transportation_mode) sorted on user_id.
- Some users may have the same number of activities tagged with e.g.
walk and car. In this case it is up to you to decide which transportation
mode to include in your answer (choose one).
- Do not count the rows where the mode is null.

In [52]:
users_most_used_mode = {}
all_users = db.UserCollection.find({})

for user in all_users:
    user_id = user['_id']
    user_activities = list(db.ActivityCollection.find({'user': user_id, 'transportation_mode': {'$ne': None}}))
    modes = [activity['transportation_mode'] for activity in user_activities]

    modes_count = Counter(modes)
    if modes_count:
        users_most_used_mode[user_id] = modes_count.most_common(1)[0][0]

print("Users and their most used transportation mode:")

for user, mode in users_most_used_mode.items():
    print(f"User: {user}, Most Used Mode: {mode}")

Users and their most used transportation mode:
User: 104, Most Used Mode: bus
User: 161, Most Used Mode: walk
User: 102, Most Used Mode: walk
User: 105, Most Used Mode: walk
User: 167, Most Used Mode: walk
User: 084, Most Used Mode: walk
User: 085, Most Used Mode: walk
User: 082, Most Used Mode: walk
User: 076, Most Used Mode: car
User: 078, Most Used Mode: walk
User: 065, Most Used Mode: bike
User: 091, Most Used Mode: walk
User: 096, Most Used Mode: bike
User: 062, Most Used Mode: bus
User: 053, Most Used Mode: walk
User: 098, Most Used Mode: walk
User: 052, Most Used Mode: bus
User: 097, Most Used Mode: bike
User: 064, Most Used Mode: walk
User: 174, Most Used Mode: car
User: 129, Most Used Mode: walk
User: 116, Most Used Mode: bike
User: 111, Most Used Mode: taxi
User: 118, Most Used Mode: car
User: 144, Most Used Mode: car
User: 175, Most Used Mode: walk
User: 126, Most Used Mode: walk
User: 110, Most Used Mode: walk
User: 128, Most Used Mode: car
User: 117, Most Used Mode: walk
U

----

In [13]:
connector.close_connection()


-----------------------------------------------
Connection to mongodb-db is closed
